In [21]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# Method 1

In [8]:
from img2table.ocr import TextractOCR
from img2table.document import Image
import os
ocr = TextractOCR()

In [9]:
list_of_files = os.listdir("Image_dataset/")

In [8]:

for file in list_of_files:
    src ="./Image_Dataset/" + file
    # Instantiation of document, either an image or a PDF
    doc = Image(src)

    dest = './extract/'+ file[:-4]+'.xlsx' # storing in extract folder

    # Extraction of tables and creation of a xlsx file containing tables
    doc.to_xlsx(dest=dest,
                ocr=ocr,
                implicit_rows=True,
                borderless_tables=True,
                min_confidence=50)

In [ ]:
# Method 2

In [9]:
import tabula

pages=[7,5,29,31,32]
# Read the PDF file
pdf_file = "spotifyQ4.pdf"

for page in pages:
# convert PDF into CSV
    tabula.convert_into(pdf_file,'./extract_2/' +str(page)+".csv", output_format="csv",pages=page )

# convert all PDFs in a directory
#tabula.convert_into_by_batch("input_directory", output_format='csv', pages='all')

In [ ]:
# Method 3

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import PIL
import pytesseract
import os

In [3]:
print(pytesseract.get_languages())



['eng', 'osd', 'snum']


In [4]:
languages_ = "eng" 

In [13]:

img_pl=PIL.Image.open('./image_dataset/'+list_of_files[0]) # You can use opencv for that option too

In [26]:
data = pytesseract.image_to_data(
                        img_pl, 
                        lang=languages_, 
                        output_type='data.frame' 
                       # config=special_config)
)

In [27]:
print(data)

     level  page_num  block_num  par_num  line_num  word_num  left  top  \
0        1         1          0        0         0         0     0    0   
1        2         1          1        0         0         0    73   79   
2        3         1          1        1         0         0    73   79   
3        4         1          1        1         1         0    73   79   
4        5         1          1        1         1         1    73   80   
5        5         1          1        1         1         2   194   80   
6        5         1          1        1         1         3   502   80   
7        5         1          1        1         1         4   725   90   
8        5         1          1        1         1         5   823   79   
9        2         1          2        0         0         0    78  192   
10       3         1          2        1         0         0    78  192   
11       4         1          2        1         1         0    78  192   
12       5         1     

In [28]:
def optimizeDf(old_df: pd.DataFrame) -> pd.DataFrame:
    df = old_df[["left", "top", "width", "text"]]
    df['left+width'] = df['left'] + df['width']
    df = df.sort_values(by=['top'], ascending=True)
    df = df.groupby(['top', 'left+width'], sort=False)['text'].sum().unstack('left+width')
    df = df.reindex(sorted(df.columns), axis=1).dropna(how='all').dropna(axis='columns', how='all')
    df = df.fillna('')
    return df

data_imp_sort = optimizeDf(data)

/var/folders/qj/rbyl_gdj7xn3ycgcxjpvn5080000gn/T/ipykernel_1772/4046707033.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['left+width'] = df['left'] + df['width']


In [31]:
def mergeDfColumns(old_df: pd.DataFrame, threshold: int = 10, rotations: int = 5) -> pd.DataFrame:
  df = old_df.copy()
  for j in range(0, rotations):
    new_columns = {}
    old_columns = df.columns
    i = 0
    while i < len(old_columns):
      if i < len(old_columns) - 1:
        # If the difference between consecutive column names is less than the threshold
        if any(old_columns[i+1] == old_columns[i] + x for x in range(1, threshold)):
          new_col = df[old_columns[i]].astype(str) + df[old_columns[i+1]].astype(str)
          new_columns[old_columns[i+1]] = new_col
          i = i + 1
        else: # If the difference between consecutive column names is greater than or equal to the threshold
          new_columns[old_columns[i]] = df[old_columns[i]]
      else: # If the current column is the last column
        new_columns[old_columns[i]] = df[old_columns[i]]
      i += 1
    df = pd.DataFrame.from_dict(new_columns).replace('', np.nan).dropna(axis='columns', how='all')
  return df.replace(np.nan, '')

def mergeDfRows(old_df: pd.DataFrame, threshold: int = 10) -> pd.DataFrame:
    new_df = old_df.iloc[:1]
    for i in range(1, len(old_df)):
        # If the difference between consecutive index values is less than the threshold
        if abs(old_df.index[i] - old_df.index[i - 1]) < threshold: 
            new_df.iloc[-1] = new_df.iloc[-1].astype(str) + old_df.iloc[i].astype(str)
        else: # If the difference is greater than the threshold, append the current row
            new_df = new_df._append(old_df.iloc[i])
    return new_df.reset_index(drop=True)
    
df_new_col = mergeDfColumns(data_imp_sort)
merged_row_df = mergeDfRows(df_new_col)


In [32]:
print(merged_row_df)

        126  173       218    252  268     290  312       359        412   \
0                                                                           
1             Key                                                           
2                                                                           
3                                                                           
4                                                                           
5                                                                           
6                                                                           
7                                                                           
8                                                                           
9                                                                           
10                                                                          
11                                                                          